In [23]:
#libraries
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import IsolationForest

In [24]:
# preparing the dataset
column_names = ["Class","Alcohol","Malic acid","Ash","Alcalinity of ash","Magnesium",
                "Total phenols","Flavanoids","Nonflavanoid phenols","Proanthocyanins",
                "Color intensity","Hue","OD280/OD315 of diluted wines","Proline"]

path = "./wine.data" 
df = pd.read_csv(path, names=column_names)

In [25]:
imputer = KNNImputer(n_neighbors=5) # more naturally preserves local data instead of median.
df.iloc[:, 1:] = imputer.fit_transform(df.iloc[:, 1:])

In [26]:
#cleaning outliers values 
iso_forest = IsolationForest(contamination=0.05, random_state=42) # more flexible solution
outliers = iso_forest.fit_predict(df.iloc[:, 1:])
df = df[outliers == 1]  

In [27]:
# scaling
scaler = StandardScaler()
df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])

In [28]:
X = df.iloc[:, 1:] 
y = df['Class'] - 1

# splitting train-test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=60, stratify=y)

In [29]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42), # the risk of overfitting is reduced
    "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
}

In [30]:
# validation with stratified fold
cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42) 

In [31]:
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"\n{name}:")
    print(f"Average CV Accuracy: {scores.mean():.4f}")
    print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")


Random Forest:
Average CV Accuracy: 0.9833
Test Accuracy: 0.9804

XGBoost:
Average CV Accuracy: 0.9750
Test Accuracy: 0.9804
